In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import csv
import random
from glob import glob
from tqdm import tqdm
from collections import namedtuple

import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch

import sys
sys.path.append("/content/gdrive/MyDrive/imc/superglue-pretrained-network")

from models.matching import SuperGlue, Matching, SuperPoint
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

In [3]:
src = '/content/gdrive/MyDrive/imc/input'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]
test_samples_df = pd.DataFrame(test_samples, columns=["sample_id", "batch_id", "image_1_id", "image_2_id"])
test_samples_df

,sample_id,batch_id,image_1_id,image_2_id
0,googleurban;1cf87530;a5a9975574c94ff9a285f58c3...,1cf87530,a5a9975574c94ff9a285f58c39b53d2c,0143f47ee9e54243a1b8454f3e91621a
1,googleurban;6ceaefff;39563e58b2b7411da3f06427c...,6ceaefff,39563e58b2b7411da3f06427c9ee4239,0303b05ca0cb46959eac430e4b2472ca
2,googleurban;d91db836;81dd07fb7b9a4e01996cee637...,d91db836,81dd07fb7b9a4e01996cee637f91ca1a,0006b1337a0347f49b4e651c035dfa0e


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
resize = [-1, ]
resize_float = True

config = {
    "superpoint": {
        "nms_radius": 4,
        "keypoint_threshold": 0.005,
        "max_keypoints": 1024
    }
}
matching = SuperPoint(config).eval().to(device)

Loaded SuperPoint model


In [18]:
F_dict = {}
for i, row in tqdm(enumerate(test_samples)):
    sample_id, batch_id, image_1_id, image_2_id = row
    
    image_fpath_1 = f'{src}/test_images/{batch_id}/{image_1_id}.png'
    image_fpath_2 = f'{src}/test_images/{batch_id}/{image_2_id}.png'
    
    image_1, inp_1, scales_1 = read_image(image_fpath_1, device, resize, 0, resize_float)
    image_2, inp_2, scales_2 = read_image(image_fpath_2, device, resize, 0, resize_float)
    
    pred = matching({'image': inp_1}) 
    #pred = {k: v[0] for k, v in pred.items()}


3it [00:05,  1.73s/it]


In [19]:
pred['descriptors']

[tensor([[ 0.1021,  0.0016,  0.0168,  ..., -0.0038, -0.0294,  0.0258],
         [-0.0134, -0.0393,  0.0581,  ...,  0.0657, -0.0456, -0.0663],
         [-0.0072, -0.1179,  0.0849,  ...,  0.0344,  0.1670,  0.1882],
         ...,
         [ 0.0238, -0.0826,  0.0567,  ...,  0.0078, -0.0719, -0.0516],
         [ 0.0241,  0.0389, -0.0948,  ..., -0.0318, -0.1112, -0.0727],
         [ 0.0679,  0.0584,  0.0193,  ...,  0.0401,  0.0378,  0.0339]],
        grad_fn=<SelectBackward0>)]

In [15]:
pred['keypoints'].shape #(2048, 2)

torch.Size([884, 2])

In [17]:
pred['scores'].shape #(2048,)

torch.Size([884])